In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

c:\Users\troyd\Programs\ECE364D\CERS\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

def load_model(
    model_name: str = "google/gemma-2b-it",
) -> tuple[AutoModelForCausalLM, AutoTokenizer]:
    config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=False,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_name, low_cpu_mem_usage=True, quantization_config=config
    )

    tokenizer = AutoTokenizer.from_pretrained(model_name)

    return model, tokenizer

In [3]:
def generate_prompt(movie: str):
    return f"""You are a movie enthusiast browsing for something to watch. You're in the mood for a specific type of film, but you can't quite put your finger on it. You want to provide the recommendation system with enough hints to suggest the perfect movie for you. The movie that you want to watch is: {movie}. Now, craft a sentence that will help the recommendation system suggest this movie to you, WITHOUT mentioning the title or any specifics like character names. Think about the genre, the tone, or any other characteristic that might help narrow down your search. Remember, the goal is to guide the recommendation engine to suggest the hidden movie you have in mind. Your response should sound conversational and not be too enthusiastic.Reply ONLY with the human-like request. DO NOT include any other text."""

In [4]:
# Load the model
model, tokenizer = load_model()

c:\Users\troyd\Programs\ECE364D\CERS\.venv\Lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\troyd\.cache\huggingface\hub\models--google--gemma-2b-it. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation func

bin c:\Users\troyd\Programs\ECE364D\CERS\.venv\Lib\site-packages\bitsandbytes\libbitsandbytes_cuda118.dll


Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.54s/it]
You are calling `save_pretrained` to a 4-bit converted model, but your `bitsandbytes` version doesn't support it. If you want to save 4-bit models, make sure to have `bitsandbytes>=0.41.3` installed.


In [13]:
# Form prompt
chat = [
    { "role": "user", "content": generate_prompt("The Matrix")},
]

prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)

# Tokenize
input_tokens = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")

# Generate
output_tokens = model.generate(input_ids=input_tokens.to(model.device), max_new_tokens=150)[0]

# Decode
response = tokenizer.decode(output_tokens, skip_special_tokens=True).split("\nmodel\n")[-1]

print(response)

I'm craving something gritty and dystopian. A film that explores the complexities of reality and the struggle for freedom, where the lines between truth and illusion blur.
